<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220107_sm_num_to_text_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification Knowcode using KoBERT Model

할일 : 텍스트로 변형 가능한 데이터 찾기

예 : 최종학력, 전공, 성별, 등

In [2]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-j1o71_17/kobert-tokenizer_bac5fdf1885144308a07b11bf0ce9988
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-j1o71_17/kobert-tokenizer_bac5fdf1885144308a07b11bf0ce9988
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_49l2ein
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_49l2ein


In [14]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
import warnings

In [15]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [43]:
warnings.filterwarnings(action='ignore') 
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]

## 2017 텍스트 데이터 전처리

In [24]:
train1

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,...,bq18_4,bq18_5,bq18_6,bq18_7,bq19,bq19_1,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,...,3,2,3,3,2,생산설비의 자동화로,2,3,5,10.0,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,없다,없다,건설현장 노무직,없다,4,1,52,2,실업,1,1,1,4000,,2200,825101
1,1,4,5,4,5,3,4,3,4,3,4,1,,1,,1,,1,,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,,2,...,4,2,2,4,2,건설 수주가 없어서,3,3,6,50.0,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,"매타기, 드라이버, 가위, CAD",,,,2,1,38,4,건축공학,1,1,1,,,2400,140204
2,2,3,4,3,4,3,4,5,6,4,5,1,,1,,3,4,3,4,3,4,1,,3,4,3,4,1,,3,4,3,4,3,4,3,4,3,4,3,...,4,2,2,4,2,신축 건설경기가 좋지 않아서,3,2,5,40.0,2,2,2,2,2,2,2,2,4,4,4,2,2,없다,"건축설계표, 오토캐드, 파이프, 보일러",,,,2,1,50,4,건축공학,1,1,1,4000,,2400,140204
3,3,3,3,3,3,3,5,4,5,4,6,3,5,3,4,4,5,5,5,4,5,3,4,4,5,3,5,3,4,4,4,3,4,2,2,4,4,4,2,4,...,3,2,2,3,1,"업무량 감소, 자동화로 인해서",4,2,6,30.0,2,1,1,2,1,1,1,1,3,4,4,4,4,지적기사,"토탈스테이션, SZP",무,지적공무원,,4,1,42,4,환경학과,1,1,1,7000,,3500,140601
4,4,4,5,3,4,3,4,4,5,3,4,1,,1,,1,,1,,3,4,2,2,3,4,3,4,1,,3,4,3,4,3,4,3,4,1,,2,...,4,2,2,4,2,부동산 거품 빠지면서 건축 경기도 덩달아 불황,3,2,3,40.0,2,2,2,2,2,2,2,2,4,3,4,3,3,없다,"캐드, 엑셀, 건축도면",,,,2,1,51,4,건축공학,1,1,1,4000,,2500,140204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,5,2,4,3,3,2,2,2,3,2,3,3,3,2,3,2,3,3,4,4,5,4,5,3,4,2,3,3,4,2,2,2,2,2,2,3,5,1,...,4,3,2,3,3,출판업무 특성상 기획업무는 변함없음,2,3,1,10.0,2,1,2,2,2,2,2,2,4,4,4,4,3,없다,"한글, 편집프로그램, 포토샵, 일러스트",없다,마케팅업무,없다,2,1,50,4,산업디자인,1,1,1,5200,,1800,411301
9482,9482,5,5,5,5,5,5,3,4,4,5,5,5,4,4,5,5,4,5,4,5,4,5,5,5,4,5,4,5,5,4,2,2,3,4,4,5,5,5,3,...,4,1,2,4,4,항공우주분야는 국가 경쟁력을 위해 육성해야 되는 분야이므로,3,3,5,20.0,2,2,1,2,2,2,2,2,4,4,3,4,3,없다,"AUTO CAD , CATIA, 전문 특수정비도구(툴)",없다,없다,없다,2,1,37,4,우주항공공학,1,1,1,4000,,3000,151105
9483,9483,3,3,4,6,3,3,4,5,4,5,3,4,3,4,1,,2,2,3,3,3,4,3,4,3,4,3,4,3,3,5,7,5,7,4,6,1,,4,...,2,4,3,3,3,건설경기는 꾸준해서,2,3,1,0.0,2,2,2,2,2,2,2,2,3,3,4,3,1,건설 노가대,"볼트, 리벳",없다,없다,없다,2,1,32,2,기계,1,4,2,2700,,1500,701101
9484,9484,3,5,3,5,4,5,3,4,3,5,4,6,4,5,4,5,4,6,4,6,3,4,3,5,3,4,2,3,4,6,3,3,3,4,3,3,4,5,4,...,3,3,2,3,3,공무원의 인원은 크게 변동이 없어서,2,2,5,20.0,2,1,1,2,1,2,1,1,3,4,4,3,2,없다,컴퓨터,없다,모름,,2,2,40,4,문헌정보학,1,1,1,6800,,2500,25402


In [26]:
train1['bq1']

0        3
1       13
2       13
3       13
4        6
        ..
9481    13
9482     3
9483     6
9484    15
9485     8
Name: bq1, Length: 9486, dtype: int64

In [7]:
def bq1_to_text(num:int):
  if num == 1:
    return '산업유형은 농업, 임업, 어업.'
  elif num == 2:
    return '산업유형은 광업.'
  elif num == 3:
    return '산업유형은 제조업.'
  elif num == 4:
    return '산업유형은 전기, 가스, 증기 및 공기조절 공급업.'
  elif num == 5:
    return '산업유형은 수도·하수·폐기물 처리, 원료재생업.'
  elif num == 6:
    return '산업유형은 건설업.'
  elif num == 7:
    return '산업유형은 도매 및 소매업.'
  elif num == 8:
    return '산업유형은 운수 및 창고업.'
  elif num == 9:
    return '산업유형은 숙박 및 음식점업.'
  elif num == 10:
    return '산업유형은 정보통신업.'
  elif num == 11:
    return '산업유형은 금융 및 보험업.'
  elif num == 12:
    return '산업유형은 부동산업.'
  elif num == 13:
    return '산업유형은 전문, 과학 및 기술 서비스업.'
  elif num == 14:
    return '산업유형은 사업시설 관리, 사업지원 및 임대 서비스업.'
  elif num == 15:
    return '산업유형은 공공행정, 국방 및 사회보장 행정.'
  elif num == 16:
    return '산업유형은 교육 서비스업.'
  elif num == 17:
    return '산업유형은 보건업 및 사회복지 서비스업.'
  elif num == 18:
    return '산업유형은 예술, 스포츠 및 여가관련 서비스업.'
  elif num == 19:
    return '산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.'
  elif num == 20:
    return '산업유형은 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동.'
  elif num == 21:
    return '산업유형은 국제 및 외국기관.'

train1['bq1'] = train1['bq1'].map(bq1_to_text)

In [9]:
def bq2_to_text(num:int):
  if num == 1:
    return '근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).'
  elif num == 2:
    return '근무업체 유형은 정부기관 및 산하기관(공기업 포함).'
  elif num == 3:
    return '근무업체 유형은 (재단, 사단)법인단체.'
  elif num == 4:
    return '근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).'
  elif num == 5:
    return '근무업체 유형은 학교법인 혹은 학원.'
  elif num == 6:
    return '근무업체 유형은 기타.'

train1['bq2'] = train1['bq2'].map(bq2_to_text)

0    근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함)
1    근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함)
2    근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함)
3        근무업체 유형은 정부기관 및 산하기관(공기업 포함)
4    근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함)
Name: bq2, dtype: object

In [10]:
def bq3_to_text(num:int):
  if num == 1:
    return '근무업체 규모는 1명 ~ 4명.'
  elif num == 2:
    return '근무업체 규모는 5명 ~ 49명.'
  elif num == 3:
    return '근무업체 규모는 50명 ~ 99명.'
  elif num == 4:
    return '근무업체 규모는 100명 ~299명.'
  elif num == 5:
    return '근무업체 규모는 300명 ~ 999명.'
  elif num == 6:
    return '근무업체 규모는 1,000명 이상.'

train1['bq3'] = train1['bq3'].map(bq3_to_text)

0     근무업체 규모는 1,000명 이상
1     근무업체 규모는 50명 ~ 99명
2     근무업체 규모는 50명 ~ 99명
3     근무업체 규모는 1,000명 이상
4    근무업체 규모는 100명 ~299명
Name: bq3, dtype: object

In [21]:
def bq5_1_to_text(num:int):
  try :
    num = int(num)
    if num == 1:
      return '훈련기간은 1개월 미만.'
    elif num == 2:
      return '훈련기간은 1개월 이상~3개월 미만.'
    elif num == 3:
      return '훈련기간은 3개월 이상~6개월 미만.'
    elif num == 4:
      return '훈련기간은 6개월 이상~1년 미만.'
    elif num == 5:
      return '훈련기간은 1년 이상~2년 미만.'
    elif num == 6:
      return '훈련기간은 2년 이상.'
  except :
    return '훈련기간은 필요없다.'

train1['bq5_1'] = train1['bq5_1'].map(bq5_1_to_text)

0    훈련기간은 1개월 이상~3개월 미만
1             훈련기간은 필요없다
2             훈련기간은 필요없다
3           훈련기간은 1개월 미만
4             훈련기간은 필요없다
Name: bq5_1, dtype: object

In [27]:
def bq6_to_text(num:int):
  if num == 1:
    return '관련된 업무경험은 1개월 미만.'
  elif num == 2:
    return '관련된 업무경험은 1개월 이상~3개월 미만.'
  elif num == 3:
    return '관련된 업무경험은 3개월 이상~6개월 미만.'
  elif num == 4:
    return '관련된 업무경험은 6개월 이상~1년 미만.'
  elif num == 5:
    return '관련된 업무경험은 1년 이상~2년 미만.'
  elif num == 6:
    return '관련된 업무경험은 2년 이상.'
  elif num == 7:
    return '이전 업무경험 필요없음.'

train1['bq6'] = train1['bq6'].map(bq6_to_text)

0      관련된 업무경험은 1년 이상~2년 미만
1            관련된 업무경험은 2년 이상
2            관련된 업무경험은 2년 이상
3    관련된 업무경험은 1개월 이상~3개월 미만
4            관련된 업무경험은 2년 이상
Name: bq6, dtype: object

In [28]:
def bq7_to_text(num:int):
  if num == 1:
    return '업무 수행 교육수준은 중학교 졸업 이하.'
  elif num == 2:
    return '업무 수행 교육수준은 고등학교 졸업.'
  elif num == 3:
    return '업무 수행 교육수준은 고등학교 졸업 후 직업교육(6개월 이상) 이수.'
  elif num == 4:
    return '업무 수행 교육수준은 전문대학교 졸업.'
  elif num == 5:
    return '업무 수행 교육수준은 대학교 졸업.'
  elif num == 6:
    return '업무 수행 교육수준은 대학원 석사 졸업.'
  elif num == 7:
    return '업무 수행 교육수준은 대학원 박사 졸업.'

train1['bq7'] = train1['bq7'].map(bq7_to_text)

0     업무 수행 교육수준은 고등학교 졸업
1      업무 수행 교육수준은 대학교 졸업
2      업무 수행 교육수준은 대학교 졸업
3    업무 수행 교육수준은 전문대학교 졸업
4      업무 수행 교육수준은 대학교 졸업
Name: bq7, dtype: object

In [29]:
def bq17_to_text(num:int):
  if num == 1:
    return '업무성과 극대화 연령대은 20대 이하.'
  elif num == 2:
    return '업무성과 극대화 연령대은 30대.'
  elif num == 3:
    return '업무성과 극대화 연령대은 40대.'
  elif num == 4:
    return '업무성과 극대화 연령대은 50대.'
  elif num == 5:
    return '업무성과 극대화 연령대은 60대 이상.'

train1['bq17'] = train1['bq17'].map(bq17_to_text)

0    업무성과 극대화 연령대은 30대
1    업무성과 극대화 연령대은 40대
2    업무성과 극대화 연령대은 40대
3    업무성과 극대화 연령대은 40대
4    업무성과 극대화 연령대은 40대
Name: bq17, dtype: object

In [31]:
def bq23_to_text(num:int):
  return '기술적 변화요인으로 인한 업무 대체 비율은 ' + str(num) + '%.'

train1['bq23'] = train1['bq23'].map(bq23_to_text)

In [34]:
def bq36_to_text(num:int):
  if num == 1:
    return '나의 성별은 남성이다.'
  elif num == 2:
    return '나의 성별은 여성이다.'

train1['bq36'] = train1['bq36'].map(bq36_to_text)

0       나의 성별은 남성이다.
1       나의 성별은 남성이다.
2       나의 성별은 남성이다.
3       나의 성별은 남성이다.
4       나의 성별은 남성이다.
            ...     
9481    나의 성별은 남성이다.
9482    나의 성별은 남성이다.
9483    나의 성별은 남성이다.
9484    나의 성별은 여성이다.
9485    나의 성별은 남성이다.
Name: bq36, Length: 9486, dtype: object

In [36]:
def bq37_to_text(num:int):
  return '나의 나이는 ' + str(num) + '살이다.'

train1['bq37'] = train1['bq37'].map(bq37_to_text)

In [38]:
def bq38_to_text(num:int):
  if num == 1:
    return '나의 최종학력은 중학교 졸업 이하이다.'
  elif num == 2:
    return '나의 최종학력은 고등학교 졸업이다.'
  elif num == 3:
    return '나의 최종학력은 2~3년제 대학교 졸업이다.'
  elif num == 4:
    return '나의 최종학력은 대학교 졸업이다.'
  elif num == 5:
    return '나의 최종학력은 대학원 석사 졸업이다.'
  elif num == 6:
    return '나의 최종학력은 대학원 박사 졸업이다.'

train1['bq38'] = train1['bq38'].map(bq38_to_text)

In [46]:
def bq40_to_text(num:int):
  try:
    num = int(num)
    if num == 1:
      return '나의 계약형태는 정규직이다.'
    elif num == 2:
      return '나의 계약형태는 비정규직이다.'
  except:
    return '나의 계약형태는 알 수 없다.'

train1['bq40'] = train1['bq40'].map(bq40_to_text)

In [50]:
def bq41_1_to_text(num:int):
  try:
    num = int(num)
    return '나의 근로소득은 ' + str(num) + '만원이다.'
  except:
    return '나의 근로소득은 알 수 없다.'

train1['bq41_1'] = train1['bq41_1'].map(bq41_1_to_text)

0       나의 근로소득은 4000만원이다.
1         나의 근로소득은 알 수 없다.
2       나의 근로소득은 4000만원이다.
3       나의 근로소득은 7000만원이다.
4       나의 근로소득은 4000만원이다.
               ...        
9481    나의 근로소득은 5200만원이다.
9482    나의 근로소득은 4000만원이다.
9483    나의 근로소득은 2700만원이다.
9484    나의 근로소득은 6800만원이다.
9485    나의 근로소득은 4000만원이다.
Name: bq41_1, Length: 9486, dtype: object

In [53]:
def bq41_2_to_text(num:int):
  try:
    num = int(num)
    return '나의 초임임금은 ' + str(num) + '만원이다.'
  except:
    return '나의 초임임금은 알 수 없다.'

train1['bq41_2'] = train1['bq41_2'].map(bq41_2_to_text)

In [57]:
def bq41_3_to_text(num:int):
  try:
    num = int(num)
    return '나의 순수입은 ' + str(num) + '만원이다.'
  except:
    return '나의 순수입은 알 수 없다.'

train1['bq41_3'] = train1['bq41_3'].map(bq41_3_to_text)

In [ ]:
#bq4_1a, bq4_1b, bq4_1c, bq5_2, bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train1[col] = train1[col].map(null_to_no)

temp = ''
for col in cols:
  temp += train1[col]
  temp += ' '

train1['text'] = temp

# train1['text'] = train1[cols[0]] + ' ' + train1[cols[1]] + ' ' + train1[cols[2]] + ' ' + train1[cols[3]] + ' ' + train1[cols[4]] + ' ' + train1[cols[5]] + ' ' + train1[cols[6]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq28_1,	bq29,	bq30,	bq31,	bq32, bq33, bq37_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq28_1',	'bq29',	'bq30',	'bq31',	'bq32', 'bq33', 'bq37_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train2[col] = train2[col].map(null_to_no)

temp = ''
for col in cols:
  temp += train2[col]
  temp += ' '

train2['text'] = temp

# train2['text'] = train2[cols[0]] + ' ' + train2[cols[1]] + ' ' + train2[cols[2]] + ' ' + train2[cols[3]] + ' ' + train2[cols[4]] + ' ' + train2[cols[5]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10,	bq20_1, bq22,	bq23,	bq24, bq27_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10',	'bq20_1', 'bq22',	'bq23',	'bq24', 'bq27_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train3[col] = train3[col].map(null_to_no)

temp = ''
for col in cols:
  temp += train3[col]
  temp += ' '

train3['text'] = temp
# train3['text'] = train3[cols[0]] + ' ' + train3[cols[1]] + ' ' + train3[cols[2]] + ' ' + train3[cols[3]] + ' ' + train3[cols[4]] + ' ' + train3[cols[5]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10, bq20_1, bq26_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10', 'bq20_1', 'bq26_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train4[col] = train4[col].map(null_to_no)

temp = ''
for col in cols:
  temp += train4[col]
  temp += ' '

train4['text'] = temp

# train4['text'] = train4[cols[0]] + ' ' + train4[cols[1]] + ' ' + train4[cols[2]]

In [ ]:
df = pd.concat([train1[['idx', 'text', 'knowcode']], train2[['idx', 'text', 'knowcode']], train3[['idx', 'text', 'knowcode']], train4[['idx', 'text', 'knowcode']]], axis=0)
df = df.reset_index(drop=True)
df

,idx,text,knowcode
0,0,자동차도장기능사 없다 없다 실무교육 생산설비의 자동화로 없다 없다 없다 건설현장 노...,825101
1,1,"건축전기설비기술사 없다 없다 없다 건설 수주가 없어서 없다 매타기, 드라이버, 가위...",140204
2,2,"건축전기설비기술사 없다 없다 없다 신축 건설경기가 좋지 않아서 없다 건축설계표, 오...",140204
3,3,"지적기사 없다 없다 측량 및 정보통신기술 교육 업무량 감소, 자동화로 인해서 지적기...",140601
4,4,건축전기설비기술사 없다 없다 없다 부동산 거품 빠지면서 건축 경기도 덩달아 불황 없...,140204
...,...,...,...
35228,35244,없다 없다 없다 없다 없다 건설 경기의 어려움으로 다소 어려움 지구과학,121105
35229,35245,없다 없다 없다 없다 없다 광고 홍보 관련 시장이 켜져서 광고학,26102
35230,35246,보험계리사 없다 없다 없다 없다 보험은 계속 필요 할 것이다 국문,31301
35231,35247,화학분석기능사 없다 없다 없다 없다 인구수명이 길어지므로서 신약개발은 갈수록 많아질...,154105


In [ ]:
def remove_ddaom(text:str):
  return str(text).replace('"', '')

df['text'] = df['text'].map(remove_ddaom)

def remove_enter(text:str):
  return text.replace('\n', ' ')

df['text'] = df['text'].map(remove_enter)

In [ ]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
my_dict = {}
for i in range(len(df['knowcode'].unique())):
  my_dict[df['knowcode'].unique()[i]] = i

In [ ]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['knowcode'].map(target_to_label)

In [ ]:
dataset_train, dataset_test = train_test_split(df[['idx', 'text', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [ ]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 256
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
next(iter(data_train))

(array([   2, 2872, 6516, 7350, 6361, 5478, 6493, 3273, 3273, 3273, 3697,
        5712, 5859, 6410, 3817, 4930, 2588,  517, 5366, 6630,  529, 5712,
        6255, 6150, 7096, 6198, 4760, 6493, 7784, 5550, 3803, 7003, 7342,
        7078, 2339, 7850, 3273, 1788, 6084,  517,   46, 2872, 7350, 7479,
        7227, 5561,  517,   46, 3563, 7461, 7202, 4297, 7706, 4898, 2272,
        5503,  285,  278,  517,   46, 1055, 3273, 3273, 5137, 5452, 7821,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [ ]:
len(df['target_label'].unique())

585

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=585,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train = []
test = []
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train.append(train_acc / (batch_id+1))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test.append(test_acc / (batch_id+1))
    # torch.cuda.empty_cache()

  0%|          | 0/3524 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.231141090393066 train acc 0.125
epoch 1 batch id 201 loss 6.563636779785156 train acc 0.0018656716417910447
epoch 1 batch id 401 loss 6.555638790130615 train acc 0.0021820448877805485
epoch 1 batch id 601 loss 6.395702362060547 train acc 0.0020798668885191347
epoch 1 batch id 801 loss 6.27824592590332 train acc 0.002184769038701623
epoch 1 batch id 1001 loss 6.368040084838867 train acc 0.0037462537462537465
epoch 1 batch id 1201 loss 6.368524551391602 train acc 0.004995836802664446
epoch 1 batch id 1401 loss 6.405701637268066 train acc 0.005620985010706638
epoch 1 batch id 1601 loss 6.338610649108887 train acc 0.005465334166146159
epoch 1 batch id 1801 loss 6.275067329406738 train acc 0.005413659078289839
epoch 1 batch id 2001 loss 6.46767520904541 train acc 0.005434782608695652
epoch 1 batch id 2201 loss 6.342113494873047 train acc 0.006190368014538846
epoch 1 batch id 2401 loss 6.471826076507568 train acc 0.006559766763848397
epoch 1 batch id 2601 loss 6.406

  0%|          | 0/881 [00:00<?, ?it/s]

epoch 1 test acc 0.015607264472190692


  0%|          | 0/3524 [00:00<?, ?it/s]

...

epoch 19 test acc 0.5512607426625588


  0%|          | 0/3524 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.13091781735420227 train acc 1.0
epoch 20 batch id 201 loss 0.17782926559448242 train acc 0.931592039800995
epoch 20 batch id 401 loss 0.8495894074440002 train acc 0.9385910224438903
epoch 20 batch id 601 loss 0.15492255985736847 train acc 0.9361480865224625
epoch 20 batch id 801 loss 0.10771330446004868 train acc 0.9388264669163545
epoch 20 batch id 1001 loss 0.262124627828598 train acc 0.9396853146853147
epoch 20 batch id 1201 loss 0.2254153937101364 train acc 0.9373438800999168
epoch 20 batch id 1401 loss 0.2062436193227768 train acc 0.9366523911491792
epoch 20 batch id 1601 loss 0.2697179913520813 train acc 0.9369144284821986
epoch 20 batch id 1801 loss 0.15946465730667114 train acc 0.9366324264297612
epoch 20 batch id 2001 loss 0.08987919986248016 train acc 0.9374687656171914
epoch 20 batch id 2201 loss 1.0386147499084473 train acc 0.9382666969559291
epoch 20 batch id 2401 loss 0.15588189661502838 train acc 0.93856726364015
epoch 20 batch id 2601 loss 0.1

  0%|          | 0/881 [00:00<?, ?it/s]

epoch 20 test acc 0.5531052375547267


In [ ]:
torch.save(model, '/content/gdrive/MyDrive/know/model2.pt')

In [ ]:
model = torch.load('/content/gdrive/MyDrive/know/model2.pt')

In [ ]:
#bq4_1a, bq4_1b, bq4_1c, bq5_2, bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1

#bq31

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test1[col] = test1[col].map(null_to_no)

temp = ''
for col in cols:
  temp += test1[col]
  temp += ' '

test1['text'] = temp

# test1['text'] = test1[cols[0]] + ' ' + test1[cols[1]] + ' ' + test1[cols[2]] + ' ' + test1[cols[3]] + ' ' + test1[cols[4]] + ' ' + test1[cols[5]] + ' ' + test1[cols[6]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq28_1,	bq29,	bq30,	bq31,	bq32, bq33, bq37_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq28_1',	'bq29',	'bq30',	'bq31',	'bq32', 'bq33', 'bq37_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test2[col] = test2[col].map(null_to_no)

temp = ''
for col in cols:
  temp += test2[col]
  temp += ' '

test2['text'] = temp

# test2['text'] = test2[cols[0]] + ' ' + test2[cols[1]] + ' ' + test2[cols[2]] + ' ' + test2[cols[3]] + ' ' + test2[cols[4]] + ' ' + test2[cols[5]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10,	bq20_1, bq22,	bq23,	bq24, bq27_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10',	'bq20_1', 'bq22',	'bq23',	'bq24', 'bq27_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test3[col] = test3[col].map(null_to_no)

temp = ''
for col in cols:
  temp += test3[col]
  temp += ' '

test3['text'] = temp
# test3['text'] = test3[cols[0]] + ' ' + test3[cols[1]] + ' ' + test3[cols[2]] + ' ' + test3[cols[3]] + ' ' + test3[cols[4]] + ' ' + test3[cols[5]]

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10, bq20_1, bq26_1

cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10', 'bq20_1', 'bq26_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test4[col] = test4[col].map(null_to_no)

temp = ''
for col in cols:
  temp += test4[col]
  temp += ' '

test4['text'] = temp

# test4['text'] = test4[cols[0]] + ' ' + test4[cols[1]] + ' ' + test4[cols[2]]

In [ ]:
df = pd.concat([test1[['idx', 'text']], test2[['idx', 'text']], test3[['idx', 'text']], test4[['idx', 'text']]], axis=0)
df = df.reset_index(drop=True)
df

,idx,text
0,0,없다 없다 없다 없다 다른것으로 대체할 수 없는 업무 없다 컴퓨터 없다 없다 없다 ...
1,1,품질관리기사 없다 없다 실무교육 제조업이 줄어들 것으로 생각되기 때문 없다 제품검사...
2,2,"없다 없다 없다 기획력, 현장경험 등 1인 미디어 증가 없다 오디션(편집프로그램),..."
3,3,없다 없다 없다 없다 선호직업 아님 없다 컴퓨터 없다 없다 없다 화학
4,4,없다 없다 없다 없다 잡지를 위한 정보나 뉴스는 고갈되지 않기 때문에 꾸준히 유지될...
...,...,...
35226,35244,원자로조종면허 없다 없다 없다 없다 전기는 일상생활에 꼭 필요한 것이라서 관련 인력...
35227,35245,없다 없다 없다 없다 온라인 설문 도입으로 테블릿을 통해 온라인으로 설문을 받는 방...
35228,35246,전자산업기사 전자기사 없다 없다 없다 미래 유망 직종 전기공학
35229,35247,건축기사 건축전기설비기술사 산업안전기사 건설 관련 직무교육(Online+Offlin...


In [ ]:
def remove_ddaom(text:str):
  return str(text).replace('"', '')

df['text'] = df['text'].map(remove_ddaom)

def remove_enter(text:str):
  return text.replace('\n', ' ')

df['text'] = df['text'].map(remove_enter)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits)

  return logits

label_answer = []
step = 0
for i in tqdm(df['text']):
  label_answer.append(predict(i))

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


  0%|          | 0/35231 [00:00<?, ?it/s]

In [ ]:
submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv') # sample submission 불러오기
submission['knowcode'] = label_answer
submission

,idx,knowcode
0,0,384
1,1,363
2,2,276
3,3,534
4,4,283
...,...,...
35226,35244,210
35227,35245,313
35228,35246,354
35229,35247,78


In [ ]:
your_dict = {v: k for k, v in my_dict.items()}

In [ ]:
def label_to_knowcode(num:int):
  return your_dict[num]

submission['knowcode'] = submission['knowcode'].map(label_to_knowcode)

In [ ]:
submission.to_csv('/content/gdrive/MyDrive/know/submission.csv', index=False, encoding='utf8')